In [2]:
import sympy as sp

from symqups import s
from symqups.objects.base import PhaseSpaceObject, qpTypePSO, alphaTypePSO
from symqups.objects.scalars import q, p, alpha, alphaD, W, mu, _Primed, _DerivativeSymbol, hbar, _DePrimed, StateFunction
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho, Operator
from symqups.utils.algebra import define, qp2a, get_random_poly
from symqups.operations.operator_ordering import sOrdering, _make_normal_ordered, explicit, express
# from symqups.operations.quantization import s_quantize, naive_quantize
from symqups.objects import scalars
from symqups.objects.cache import _sub_cache
from symqups.operations.star_product import Bopp, Star, _replace_diff
from symqups.utils.multiprocessing import _mp_helper
from symqups.utils._internal._basic_routines import _operation_routine
from symqups.utils._internal._operator_handling import (
    _separate_operator,
    _separate_term_by_polynomiality,
    _separate_term_oper_by_sub,
    _collect_alpha_type_oper_from_monomial_by_sub,
    _get_oper_sub
)

a = alpha(0)
a_1 = alpha(1)
aop = annihilateOp(0)
aop_1 = annihilateOp(1)
ad = alphaD(0)
ad_1 = alphaD(1)
adop = createOp(0)
adop_1 = createOp(1)
x = sp.Symbol("x")
qq = q(0)
pp = p(0)
W = scalars.W
s.val = 0
s

s

In [ ]:
nc_part = sp.Mul.flatten([adop_1, aop**2, rho, aop])[1]

from copy import deepcopy
from typing import Tuple

# A universal Operator expression
# is indicated by having at least one Operator with `has_sub=False`, 
# We cannot reorder universally noncommuting expressions. 

def _is_universal(A : sp.Expr) -> bool:
    return not(all(atom.has_sub for atom in A.atoms(Operator)))


reordered_nc_part = []
reorderable_nc = []

def _treat_reorderable_nc():
    global reorderable_nc
    nc_sub_lst = [_get_oper_sub(nc) for nc in reorderable_nc]

    used_nc_idx = []
    for sub in _sub_cache:
        # Since _sub_cache (as well as Scalar) is ordered according to
        # sympy's canon, we try to do the same for Operator, the ordering
        # of which sympy does not automatically do due to the noncommuting
        # nature.
        
        for j, nc_sub in enumerate(nc_sub_lst):
    
            if (sub in nc_sub) and (j not in used_nc_idx):
                
                # Using this algorithm, non-Operator objects are added
                # into the argument sequence as soon as they are encountered
                # (during the j=0 iteration) so it follows 
    
                reordered_nc_part.append(reorderable_nc[j])
                used_nc_idx.append(j)
                
    assert len(used_nc_idx) == len(reorderable_nc)
    
    reorderable_nc = []

for nc in nc_part:
    if not(_is_universal(nc)):
        # As long as 'nc' is universal 
        reorderable_nc.append(nc)
    else:
        _treat_reorderable_nc(reorderable_nc)
        reordered_nc_part.append(nc)
_treat_reorderable_nc(reorderable_nc) # leftovers from the loop

reordered_nc_part

[\hat{a}_{0}**2,
 \hat{a}^{\dagger}_{1},
 \rho_{},
 \hat{a}_{0}**2,
 \hat{a}_{0},
 \hat{a}^{\dagger}_{1}]